In [1]:
from bs4 import BeautifulSoup as bs
import requests
from urllib.request import urlopen as uReq 
import datetime
import pandas as pd
import json
import math
import time
import os
#import logging
#import random

In [18]:
product_ids = []
f = open('list_all')
for i in f:
    product_ids.append(i.strip())

In [5]:
headers = {
    'authority': 'www.amazon.com',
    'cache-control': 'max-age=0',
    'rtt': '400',
    'downlink': '1.45',
    'ect': '3g',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'en,zh-CN;q=0.9,zh;q=0.8',
    'cookie': 'session-id=138-7828792-9054417; session-id-time=2082787201l; ubid-main=132-7497391-3364111; lc-main=en_US; at-main=Atza^|IwEBIGOMGdG-q5DtKPcrfTohUYTNBqxwshnGZO3f5ED9FxFbu6608XuXDEo1longOgpcphN_ettJDxLRAOGrpuH1AIZoiRMeujM2_eT0fyIflcdrseiEXvzsIZKzN0GEKabiy7Ubnn9ic4Gc7o5TerywdL_ixKfUSt200a8aLpzzJTKdi8tM__CDGgBTTX2DUyayGs-rain8SZIUkvcFAOwp05IFZpIj-S5yuPq_Uwf-kGuzPMSrKwzxP7JPjTjbOReS6TtYpkrIR-dVCfIqa5co0xwL7mwC0j-Qjp55A_rzxgXn5R_3zfY0xDIQ08ZB87gBrgwWWh8jN3A4QmHFpjToIu1UP3nFrBsL7czJF9JluLMdi5js4S_-DGhs793wL_DNB9CzVhWbTDALyiQF3-ZU_BHh; sess-at-main=^\\^7HA0qFRhT/k66DsIXxMlwMBmG/NeRYnvY8ctAa9TySE=^\\^; sst-main=Sst1^|PQHHD0bSOdCRONY5gD3fR8tkC9MFAzPtoqrO28r7cpf3tlQvFR-AEyaVWSf4O7xcaXNYPmT3qoXf8i8xlhIWwVXkd2XJwOLH6imoUCsbrtMCUkcReN0LRUBvWbV5aAaLqLG_mWKUEjX0DY83AFrw1HI4G4kqBdwxt_DW1NCjugUCcHofDgj5khRA5p6nDOaAfc86RQiGm5MpSztBQ0WO-qNeU4DhBg2kw-kDbVCbzlK0O9m4mS2eSRWpF-qErKyRm2r1kQFH16Bg3wn0PqK0gli9e5QHUxe-5Jxbfz1qcFcS16vGWFIN83MdqBhEY6IcU50cX5Tk1kb_QZWEG3800p8jYQ; i18n-prefs=USD; x-main=^\\^IXTSFwjYOiRMdDimJmD1oKutMwXIK2sF4mKbpZTdrjQV^@I4tf^@hBS0HSY1Q^@TqtD^\\^; session-token=^\\^/nPO8Wohk/FnQ91upK1PxcpBYKsJLO6iQ4vHpPyRRZh3YpsRK3rFlBo3UF563C3iQwydXDzhcDbZ5buPMNMTGUxuQy6NcuvyNDjofYZkBJLAHDhnHpfO92AFq9z4Wz5bJKWl4KU8LllGwaxawFX8wk0iYTjy3T+6AIwF01R171+oz+9yuZrNqZbOR2DEUZzrw6fW9VmH8Arwi8iAETUMWg==^\\^; skin=noskin; csm-hit=adb:adblk_no^&t:1613032783198^&tb:s-1PY5XBHZDW9RP3J1BP26^|1613032782736',
}


In [19]:
#product_ids = pd.read_csv('product_id_list.csv')
#product_ids = []
#f = open('list_all')
#for i in f:
    #product_ids.append(i.strip())
unscraped_id = list()
#for product_id in product_ids['0']:
for product_id in product_ids:
#for product_id in unscraped:
    amazon_all_review_lst = list()
    link = f'https://amazon.com/product-reviews/{product_id}/ref=cm_cr_arp_d_paging_btm_1?ie=UTF8&reviewerType=all_reviews&showViewpoints=1&sortBy=helpful&pageNumber=1'
    try:
        page = requests.get(link,headers=headers)
        soup = bs(page.content,'html.parser')
        reviews_n = soup.find('div',{'data-hook':'cr-filter-info-review-rating-count'})
        for i in range(3):
            if reviews_n == None:
                reviews_n = soup.find('div',{'data-hook':'cr-filter-info-review-rating-count'})
        if reviews_n == None:
            unscraped_id.append(product_id)
            continue
        reviews_number = reviews_n.get_text().strip().split('|')[1].replace('global reviews','').strip().replace(',','')
        max_pges = math.floor(int(reviews_number)/10)
        print(f'The number of pages to scrap for product {product_id} are {max_pges}')
    except Exception:
        continue
    #if max_pges < 3000:
        #continue
    for k in range(1,max_pges):
        #link = f'https://www.amazon.com/Apple-iPhone-Locked-Carrier-Subscription/product-reviews/B00006IF67/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber={k}'
        link = f'https://amazon.com/product-reviews/{product_id}/ref=cm_cr_arp_d_paging_btm_1?ie=UTF8&reviewerType=all_reviews&showViewpoints=1&sortBy=helpful&pageNumber={k}'
        try:
            page = requests.get(link,headers=headers)
        except Exception:
            continue
        soup = bs(page.content,'html.parser')
        reviews = soup.findAll('div',{'data-hook': 'review'})
        print(f'Scraping for product {product_id} at page {k}')
        for review in reviews:
            try:
                rating = review.find(attrs={'data-hook': 'review-star-rating'}).attrs['class'][2].split('-')[-1].strip()
                body = review.find(attrs={'data-hook': 'review-body'}).text.strip()
                title = review.find(attrs={'data-hook': 'review-title'}).text.strip()
                author_url = review.find(attrs={'data-hook': 'genome-widget'}).find('a', href=True)
                author = review.find(attrs={'data-hook': 'genome-widget'}).find('span', 'a-profile-name').text.strip()
                review_url = review.find(attrs={'data-hook': 'review-title'}).attrs['href']
                review_date = review.find(attrs={'data-hook': 'review-date'}).text.strip()
                verified = review.find(attrs={'data-hook': 'avp-badge'}) or ''
                types = review.find(attrs={'data-hook': 'format-strip'}) or ''
                if types != '':
                    types = types.text.strip()
                if verified != '':
                    verified = verified.text.strip()
                if author_url:
                    author_url = author_url['href'].strip()
                try:
                    helpful = review.find(attrs={'data-hook': 'helpful-vote-statement'}).text.strip()
                    helpful = helpful.strip().split(' ')[0]
                except:
                    helpful = ''
            except Exception:
                continue
        
            amazon_all_review_lst.append({'title': title,
                                'rating': rating,
                                'verified':verified,
                                'types':types,
                                'body': body,
                                'product_id':product_id,
                                'author_url': author_url,
                                'author':author,
                                'review_url': review_url,
                                'review_date': review_date,
                                'helpful':helpful,
                               }) 
    #The file where you store each product_id reviews. You add put them in a specific folders called for example 'reviews'
    #To do so, just add f'./reviews/{product_id}.json'
    with open(f'{product_id}.json','w') as jsonfile:
        json.dump(amazon_all_review_lst,jsonfile, indent=4, sort_keys=True, default=str)

0